In [8]:
import zipfile

kmz_file_path = 'C:/Users/elfak/Downloads/LISTE VILLAGES (1).kmz'
extracted_folder_path = 'C:/Users/elfak/Downloads/'

with zipfile.ZipFile(kmz_file_path, 'r') as kmz:
    kmz.extractall(extracted_folder_path)


In [3]:
import xml.etree.ElementTree as ET
import pandas as pd

# Define the KML namespace
namespace = "{http://www.opengis.net/kml/2.2}"

# Parse the KML data
kml_file_path = 'C:/Users/elfak/Downloads/doc.kml'  # Replace with your actual path
tree = ET.parse(kml_file_path)
root = tree.getroot()

# Initialize lists to store the extracted data
names = []
coordinates = []

# Extract data from Placemark elements using the namespace
for placemark in root.findall(f'.//{namespace}Placemark'):
    name = placemark.find(f'.//{namespace}name').text.strip()
    coordinates_text = placemark.find(f'.//{namespace}coordinates').text.strip()
    # Split the coordinates string into latitude, longitude, and altitude
    coordinates_list = coordinates_text.split(',')[:2]
    longitude, latitude = map(float, coordinates_list)
    
    names.append(name)
    coordinates.append((latitude, longitude))

# Create a DataFrame
data = {'name': names, 'latitude': [coord[0] for coord in coordinates], 'longitude': [coord[1] for coord in coordinates]}
df = pd.DataFrame(data)

# Save the data to an Excel file
excel_file_path = 'locations.xlsx'  # Adjust the file path as needed
df.to_excel(excel_file_path, index=False)

print(f'Data saved to {excel_file_path}')


Data saved to locations.xlsx


In [4]:
df_location = pd.read_excel('locations.xlsx')
df_location

,name,latitude,longitude
0,Douar Tazloumt,30.808782,-8.606285
1,Talat N'Yaaqoub,30.993130,-8.235383
2,Ijoukak,30.997324,-8.162536
3,دوار امندار‎,30.847017,-8.579649
4,دوار تجكَالت‎,31.060757,-8.855264
...,...,...,...
328,مجموعة مدارس تكنزا - وحدة أنومس‎,30.875672,-8.787370
329,مسجد انادن,30.983637,-8.152778
330,Agdim,31.306409,-6.854532
331,Tazelga,31.237979,-7.879984


In [ ]:
import pandas as pd
from sources.utils import update_excel, merge_by_distance_coords


In [ ]:
df = pd.read_excel('earthquake_relief.xlsx')
df_location = pd.read_excel('locations.xlsx')

df_merged = merge_by_distance_coords(df,df_location,5)
df_merged # .to_excel('temp_csv.xlsx')